In [ ]:
import os
import re
import base64
import mimetypes
import configparser
from huggingface_hub import InferenceClient
# from langchain_groq import ChatGroq
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
import base64

# # API Keys
# groq_api_key = config["groq"]["api_key"]
# groq_model = config["groq"]["model_name"]
hf_token = ""


c:\Users\sayan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
ocr_instruction_text = """


You are an OCR expert. Analyze the provided image and extract all invoice-related table data (e.g., item details, descriptions, quantities, prices, totals).  
The table format may vary and may have merged cells or irregular layouts.

Return only valid JSON in this structure:
{
  "invoice_tables": [
    {
      "headers": [string, ...],
      "rows": [
        [string, ...]
      ]
    }
  ]
}

Rules:
- Preserve text exactly as seen, fixing obvious OCR errors when confident.
- Do not add extra commentary or formatting.
- If no table is found, return: {"invoice_tables": []}

"""


In [3]:
import os
import re
import base64
import mimetypes
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
        base64_string = base64.b64encode(image_data).decode('utf-8')
        mime_type, _ = mimetypes.guess_type(image_path)
        mime_type = mime_type or "image/jpeg"
        return f"data:{mime_type};base64,{base64_string}"


In [4]:
def ocr(local_image_path):
    base64_image = image_to_base64(local_image_path)
    client = InferenceClient(api_key=hf_token)

    completion = client.chat.completions.create(
        model="Qwen/Qwen2.5-VL-7B-Instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": ocr_instruction_text
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": base64_image}
                    }
                ]
            }
        ]
    )

    return completion.choices[0].message["content"]

In [6]:
try:
    ans = ocr("table.png")
except Exception as E:
    print(E)

In [8]:
print(ans)

```json
{
  "invoice_tables": [
    {
      "headers": ["#", "Description", "Qty", "Units", "Unit price (EUR)", "Total (EUR)"],
      "rows": [
        ["1", "Projecting - Context menu for invoices list", "x1.0", "hours", "50.00", "50.00"],
        ["2", "Develop - Invoice number format template - [PRO] Duplicating invoices - Language support - Context menu for invoices list", "x17.0", "hours", "40.00", "680.00"],
        ["3", "Analysis - [PRO] Duplicating invoices - Language support", "x3.0", "hours", "35.00", "105.00"]
      ]
    }
  ]
}
```


In [13]:
from PIL import Image

# Load image
img = Image.open(r"table.png")

# Display image
img.show()


In [12]:
try:
    ans = ocr("table1.jpg")
    print(ans)
except Exception as E:
    print(E)

```json
{
  "invoice_tables": [
    {
      "headers": ["#", "PRODUCT DESCRIPTION", "PRICE", "QTY", "TOTAL"],
      "rows": [
        ["1", "Description 1", "$30.50", "3", "$91.50"],
        ["2", "Description 2", "$50.00", "1", "$50.00"],
        ["3", "Description 3", "$70.00", "5", "$350.00"],
        ["4", "Description 4", "$20.00", "2", "$40.00"],
        ["5", "Description 5", "$30.00", "3", "$90"],
        ["6", "Description 6", "$150.00", "1", "$150.00"],
        ["7", "Description 7", "$70.50", "5", "$352.50"],
        ["8", "Description 8", "$10.00", "2", "$20.00"],
        ["9", "Description 9", "$25.00", "6", "$150.00"]
      ]
    }
  ]
}
```
